This notebook trains and tests an SCA agent on the Laser Hockey Enviroment

In [1]:
# Imports
import numpy as np
import laserhockey.hockey_env as h_env
from importlib import reload
from SAC import Agent
import matplotlib.pyplot as plt

In [2]:
# Global variables
train_games = 20000 # 720/34min
test_games = 100
load_checkpoint = False
train = True
# render testing
render = False

In [3]:
# Initialization 
np.set_printoptions(suppress=True)
reload(h_env)

# get enviroment
env = h_env.HockeyEnv(mode=h_env.HockeyEnv.NORMAL)

#init agent
agent = Agent(input_dims=env.observation_space.shape, 
              env=env,
              lr = 0.0005,
              gamma = 1,
              reward_abs = True)

# and oponent
opponent = h_env.BasicOpponent()

/Users/emilbreustedt/anaconda3/envs/RL/lib/python3.8/site-packages/gymnasium/envs/registration.py:693: UserWarning: WARN: Overriding environment Hockey-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/Users/emilbreustedt/anaconda3/envs/RL/lib/python3.8/site-packages/gymnasium/envs/registration.py:693: UserWarning: WARN: Overriding environment Hockey-One-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


In [4]:
# Plotting function
def plot_learning_curve(x, scores, figure_file):
    running_avg = np.zeros(len(scores))
    for i in range(len(running_avg)):
        running_avg[i] = np.mean(scores[max(0, i-100):(i+1)])
    plt.plot(x, running_avg)
    plt.xlabel("Episodes")
    plt.ylabel("Running average of previous 100 scores")    
    plt.savefig(figure_file)

In [5]:
# Training of agent

best_score = env.reward_range[0]
score_history = []

if train:
    # conttinue from checkpoint
    if load_checkpoint:
        agent.load_models()

    for i in range(train_games):
        observation, info = env.reset()
        obs_opponent = env.obs_agent_two()

        done = False
        score = 0

        while not done:
            action1 = agent.act(observation)
            action2 = opponent.act(obs_opponent)
            
            observation_, reward, done, _, info = env.step(np.hstack([action1,action2]))
            score += reward
            agent.remember(observation, action1, reward, observation_, done)
            
            agent.learn()
                
            obs_opponent = env.obs_agent_two()
            observation = observation_
            
        score_history.append(score)
        avg_score = np.mean(score_history[-100:])
        
        # remember best model after initial fluctuation
        if avg_score > best_score and i > 500:
            best_score = avg_score
            agent.save_models()

        print('episode ', i, 'score %.1f' % score, 'avg_score %.1f' % avg_score)

    x = [i+1 for i in range(train_games)]
    plot_learning_curve(x, score_history, '10000_abs_rewward.png')

episode  0 score -11.2 avg_score -11.2
episode  1 score 7.7 avg_score -1.8
episode  2 score -11.0 avg_score -4.9
episode  3 score 8.0 avg_score -1.6
episode  4 score -10.9 avg_score -3.5
episode  5 score 6.5 avg_score -1.8
episode  6 score -13.5 avg_score -3.5
episode  7 score 9.0 avg_score -1.9
episode  8 score -18.5 avg_score -3.8
episode  9 score -27.1 avg_score -6.1
episode  10 score -11.2 avg_score -6.6
episode  11 score -19.5 avg_score -7.6
episode  12 score -11.6 avg_score -7.9
episode  13 score -50.2 avg_score -11.0
episode  14 score -14.2 avg_score -11.2
episode  15 score -24.9 avg_score -12.0
episode  16 score -11.9 avg_score -12.0
episode  17 score -24.9 avg_score -12.7
episode  18 score -13.7 avg_score -12.8
episode  19 score -12.8 avg_score -12.8
episode  20 score -11.0 avg_score -12.7
episode  21 score -37.2 avg_score -13.8
episode  22 score -11.3 avg_score -13.7
episode  23 score -36.7 avg_score -14.7
episode  24 score -6.8 avg_score -14.4
episode  25 score -53.4 avg_sco

In [ ]:
# Testing

wins = 0
losses = 0
draws = 0

agent.load_models()
opponent = h_env.BasicOpponent()

for i in range(test_games):
    if render:
        env.render() 
    observation, info = env.reset()
    obs_opponent = env.obs_agent_two()
    done = False
    while not done:
        if render:
            env.render() 
        action1 = agent.act(observation)
        action2 = opponent.act(obs_opponent)

        observation, reward, done, _, info = env.step(np.hstack([action1,action2]))

        obs_opponent = env.obs_agent_two()
    if info['winner']==1:
            wins+=1
    elif info['winner']==-1:
            losses+=1
    else:
        draws+=1
        
wins, losses, draws

.... loading models ....


(10, 5, 10)

In [ ]:
import pickle

with open("10000_abs_reward", "wb") as fp:   #Pickling
    pickle.dump(score_history, fp)

In [ ]:
with open("basis/10000_abs_reward", "rb") as fp:   # Unpickling
    score_history = pickle.load(fp)

plot_learning_curve(range(len(score_history)), score_history, 'SCA_basis.png')

FileNotFoundError: [Errno 2] No such file or directory: 'basis/10000_abs_reward'